In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("Atividade2").getOrCreate()
volumes_path = "/Volumes/atividade_2_rocketlab/landing/volumes/"
for volume in dbutils.fs.ls(volumes_path):
    print(volume.name)

In [0]:
dict_translate_names = {
    'customers': 'consumidores',
    'geolocation': 'geolocalizacao',
    'orders': 'pedidos',
    'order_items': 'itens_pedidos',
    'order_payments': 'pagamentos_pedidos',
    'order_reviews': 'avaliacoes_pedidos',
    'products': 'produtos',
    'sellers': 'vendedores',
    'product_category_name_translation': 'categorias_produtos_traducao'
}
def ingest_all_volumes_to_bronze(volumes_path, catalogo, bronza_scheme):
    for volume in dbutils.fs.ls(volumes_path):

        df = spark.read.csv(volume.path, header=True, inferSchema=True)
        # Validação: arquivo vazio
        if df.count() == 0:
            raise ValueError(f"O arquivo {volume.name} está vazio ou não pôde ser lido.")
        
        df_with_metadata = df.withColumn("data_ingestao", F.current_timestamp()).withColumn("origem", F.lit('kaggle'))

        # Salvando no bronze (simplificando o nome)
        name = volume.name.split(".")[0] # remove .csv
        name = [part for part in name.split("_") if part not in ['olist', 'dataset']]
        name = dict_translate_names.get('_'.join(name))
        table_name = 'ft_' + name
        if name == 'categorias_produtos_traducao':
            table_name = 'dm_' + name
        df_with_metadata.write.mode("overwrite").option("overwriteSchema", "true").format('delta').saveAsTable(f"{catalogo}.{bronza_scheme}.{table_name}")

       

volumes_path = "/Volumes/atividade_2_rocketlab/landing/volumes/"
catalogo = 'atividade_2_rocketlab'
bronza_scheme = 'bronze'
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalogo}.{bronza_scheme}")    
ingest_all_volumes_to_bronze(volumes_path, catalogo, bronza_scheme)

Cotação do dolar

In [0]:
import requests
import pandas as pd
from datetime import datetime, timedelta


def get_cotacao_dolar(data_inicio, data_fim):

     formato = "%d-%m-%Y"
     data_inicial = datetime.strptime(data_inicio, formato)
     data_final = datetime.strptime(data_fim, formato)

     diferenca = data_final - data_inicial

     atual = data_inicial
     cotacoes = []
     datas = []
     while atual + timedelta(days=100) <= data_final:
          
          final = atual + timedelta(days=100)
        
          url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='{atual.strftime('%m-%d-%Y')}'&@dataFinalCotacao='{final.strftime('%m-%d-%Y')}'&$top=100&$format=json&$select=cotacaoCompra,dataHoraCotacao"
          response = requests.get(url)
          dados = response.json()
          cotacoes += dados['value']
          atual += timedelta(days=101)
     
     diferenca = atual - data_inicial
     if diferenca:
          url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='{atual.strftime('%m-%d-%Y')}'&@dataFinalCotacao='{data_inicial.strftime('%m-%d-%Y')}'&$top=100&$format=json&$select=cotacaoCompra,dataHoraCotacao"
          response = requests.get(url)
          dados = response.json()
          cotacoes += dados['value']
     return cotacoes

#mes-dia-ano
data_inicio = '01-01-2016'
data_fim = '02-01-2020'
t = spark.createDataFrame(pd.DataFrame(get_cotacao_dolar(data_inicio, data_fim)))
t = (t
     .withColumn('origem', F.lit('API_BANCO_CENTRAL'))
     .withColumn('data_ingestao', F.current_timestamp())
     )
t.write.mode("overwrite").option("overwriteSchema", "true").format('delta').saveAsTable(f"{catalogo}.{bronza_scheme}.dm_cotacao_dolar")
t.display()